In [ ]:
# notebooks/3_feature_eng.ipynb

import pandas as pd
import numpy as np

# 1. Veriyi Oku (EDA'da kaydettiğimiz veya işlediğimiz halini)
df = pd.read_csv('../data/atm_data.csv')
df['transactionTime'] = pd.to_datetime(df['transactionTime'])
df.sort_values(by=['atmName', 'transactionTime'], inplace=True)

# index'i resetleyelim
df.reset_index(drop=True, inplace=True)

# 2. Tarihsel Özellikler (Date Features)
df['hour'] = df['transactionTime'].dt.hour
df['day_of_week'] = df['transactionTime'].dt.dayofweek # 0: Pazartesi, 6: Pazar
df['day_of_month'] = df['transactionTime'].dt.day
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# 3. LAG FEATURES (Gecikmeli Değişkenler - EN ÖNEMLİ KISIM)
# Modelin "hafızası" olması için geçmiş değerleri sütun olarak ekliyoruz.
# Her ATM grubu (atmName) içinde işlem yapmalıyız.

# Lag 1: Bir önceki satırdaki (genelde 1 saat önceki) çekim miktarı (totalOutcome)
# Ancak veri setinde saatler ardışık mı emin olmalıyız, basitlik adına shift kullanıyoruz.
df['lag_1'] = df.groupby('atmName')['totalOutcome'].shift(1)

# Lag 24: Tam 24 saat önceki (Dün aynı saat) çekim miktarı -> ÇOK DEĞERLİDİR
df['lag_24'] = df.groupby('atmName')['totalOutcome'].shift(24)

# 4. Rolling Window Features (Hareketli Ortalamalar)
# Son 3 saatin ortalaması
df['rolling_mean_3'] = df.groupby('atmName')['totalOutcome'].transform(lambda x: x.shift(1).rolling(window=3).mean())

# Son 24 saatin ortalaması (Trendi yakalar)
df['rolling_mean_24'] = df.groupby('atmName')['totalOutcome'].transform(lambda x: x.shift(1).rolling(window=24).mean())

# 5. Eksik Verileri Temizle (Lag aldığımız için ilk satırlar NaN olur)
print("Öncesi NaN Sayısı:", df.isnull().sum().sum())
df_model = df.dropna()
print("Sonrası NaN Sayısı:", df_model.isnull().sum().sum())

# 6. İşlenmiş Veriyi Kaydet (Model notebook'unda kullanacağız)
df_model.to_csv('../data/atm_processed.csv', index=False)
print("Feature Engineering tamamlandı! Veri kaydedildi: data/atm_processed.csv")
print(df_model[['transactionTime', 'totalOutcome', 'lag_1', 'lag_24']].head())